In [1]:
import sys
import zipfile
import os

import numpy as np
import pandas as pd
from IPython.display import display, HTML

sys.path.append("../../libs/")

from datasetutil import gerar_estatiscas_df, substituir_valores_nulos , verify_gg4cc , save_zip

In [2]:

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Metodos

In [65]:
def read(zinput,nome_arquivo,
         file_format=".csv",
         separador=',',codificacao="iso-8859-1"):
    '''
        Ler zip com os datasets
    '''
    zf = zipfile.ZipFile(zinput) 

    for file in zipfile.ZipFile.namelist(zf):
        if file == nome_arquivo+file_format:
            df = pd.read_csv(zf.open(file), header=0, encoding=codificacao,
                             sep=separador,error_bad_lines=False)#,
#             chunksize=640000000,
#             nrows=15)
            return df
# df.iloc[:, [68,2,10,11,9,28,29,13,14,15,1,16]]

# Produtos

## Abt-buy

Leipzig group

In [85]:
ds_files = 'D:\Dados\OneDrive\Doutorado\Datasets\Leipzig\\'

inputZip = 'Abt-Buy.zip'
ds1_name = 'Abt'
ds2_name = 'Buy'
gs_name ='abt_buy_perfectMapping'


zip_file = 'processed_abt-buy.zip'

ds1 = read(ds_files+inputZip,ds1_name)
ds2 = read(ds_files+inputZip,ds2_name)
gs = read(ds_files+inputZip,gs_name)

### Estatisticas

In [86]:
ds1.pop('description')
ds1.pop('price')
ds2.pop('description')
z = ds2.pop('price')
del z

In [87]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2]).to_csv(ds_files+'stats_abt-buy.csv',sep=';')

Dataset para deduplicação com:

In [88]:
display(sdf1)
display(sdf2)

,name,ds
metrica,,
count,1081,Abt
unique,1081,Abt
top,Panasonic NNSD797S Stainless Steel Countertop ...,Abt
freq,1,Abt
nan,0,Abt


,name,manufacturer,ds
metrica,,,
count,1092,1086,Buy
unique,1079,116,Buy
top,LG 5.6 cu.ft. Freestanding Electric Range,Sony,Buy
freq,5,170,Buy
nan,0,6,Buy


### Problemas nos dados


sem valores nulos

In [89]:
#df = substituir_valores_nulos(df)

In [90]:
gs.columns = ['id1','id2']
da, db = verify_gg4cc(gs)

O gabarito não é de um dataset clean-clean

In [91]:
display(da.head(5))
display(db.head(5))

,id1,id2
40,32625,201935116
41,32625,201935171
61,22042,202145734
62,22042,202238311
66,23006,202456880


,id1,id2
82,23651,202784323
83,37360,202784323
510,32628,207554111
511,32630,207554111
601,34815,208033806


### Solução do problema

In [92]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

Removendo as entidades do DS

In [93]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

Limpando o gabarito

In [94]:
gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

In [95]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dados limpos, agora salvando

In [96]:
save_zip(cds1,cds2,gs,estatiscas,ds_files + zip_file)

# Citacoes

## DBLP-ACM

In [104]:
ds_files = 'D:\Dados\OneDrive\Doutorado\Datasets\Leipzig\\'

inputZip = 'DBLP-ACM.zip'
ds1_name = 'DBLP2'
ds2_name = 'ACM'
gs_name ='DBLP-ACM_perfectMapping'


zip_file = 'processed_DBLP-ACM.zip'

ds1 = read(ds_files+inputZip,ds1_name)
ds2 = read(ds_files+inputZip,ds2_name)
gs = read(ds_files+inputZip,gs_name)

### Limpando

In [105]:
ds1.pop('title')
z = ds2.pop('title')
del z

### Estatisticas

In [106]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2]).to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,id,authors,venue,ds
metrica,,,,
count,2616,2616,2616,DBLP2
unique,2616,2316,5,DBLP2
top,conf/sigmod/PangCL94,Richard T. Snodgrass,VLDB,DBLP2
freq,1,25,877,DBLP2
nan,0,0,0,DBLP2


,authors,venue,ds
metrica,,,
count,2280,2294,ACM
unique,2008,5,ACM
top,Karl Aberer,International Conference on Management of Data,ACM
freq,16,797,ACM
nan,14,0,ACM


### Limpando os dados

Sem valores nulos

In [107]:
gs.columns = ['id1','id2']
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

### Salvando os dados

In [108]:
save_zip(cds1,cds2,gs,estatiscas,ds_files + zip_file)